In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
from gensim.models import Word2Vec

### load data

In [2]:
train_x = pd.read_pickle('E:\\school\\2-2\\AI\\Project2\\data\\Modified_LTD_x.pkl')
train_y = pd.read_pickle('E:\\school\\2-2\\AI\\Project2\\data\\Modified_LTD_y.pkl')

In [3]:
train_x.head()

0    [[stuff, going, moment, mj, started, listening...
1    [[classic, war, worlds, timothy, hines, entert...
2    [[film, starts, manager, nicholas, bell, givin...
3    [[must, assumed, praised, film, greatest, film...
4    [[superbly, trashy, wondrously, unpretentious,...
Name: review, dtype: object

In [4]:
train_y.head()

0    1
1    1
2    0
3    0
4    1
Name: sentiment, dtype: int64

In [5]:
model = Word2Vec.load('E:\\school\\2-2\\AI\\Project2\\data\\model_f')

### average word_2_vector

In [6]:
train_data_x = []
index2word_set = set(model.wv.index2word)
for review in train_x:
    cnt = 0
    vec = np.zeros((1, 100))
    for sentence in review:
        for word in sentence:
            if word in index2word_set:
                vec = vec + model.wv.__getitem__(word)
                cnt = cnt+1
    vec = vec/cnt
    train_data_x.extend(vec)

In [7]:
train_data_x = np.array(train_data_x)
train_data_x = np.nan_to_num(train_data_x)

In [8]:
train_data_x[1]

array([-3.09661818e-01,  1.78949442e-01,  2.26472974e-01, -1.86070921e-01,
        1.95811458e-01,  4.38611245e-01, -4.08524124e-01, -6.65671946e-01,
        2.63174426e-01, -1.33695632e-01, -1.07492029e-01,  1.55464631e-02,
       -3.09734471e-01, -4.25791129e-01, -7.08653548e-01,  5.44282131e-02,
        1.56599880e-01,  2.75622610e-01, -1.80435793e-01,  6.55734103e-02,
       -8.44379794e-02, -2.69451454e-01,  2.14453976e-01,  7.74724223e-02,
       -1.75981418e-01, -6.13535655e-02,  9.79267536e-02, -1.64134563e-01,
        1.76172013e-01, -5.81104011e-01,  8.90994138e-01, -7.53316830e-02,
        3.52008596e-01, -5.03559722e-01, -2.82672584e-01,  9.93822188e-02,
       -2.81171313e-01,  1.32458088e-01,  1.92223500e-01, -4.80042285e-01,
        2.56657893e-01,  6.48274922e-04, -1.41304082e-01, -3.52795055e-01,
       -3.32541964e-01,  2.56887954e-02,  8.06368992e-01, -2.29979576e-01,
        3.17670996e-01, -3.03196477e-01,  2.99626871e-02,  1.50498620e-01,
       -3.57288370e-01, -

In [9]:
train_y[1]

1

In [10]:
train_data_y = np.array(train_y)

### SVM with 10 KFold

In [11]:
import sklearn.svm as svm
import random
from sklearn.model_selection import StratifiedKFold 

In [12]:
clf = svm.SVC()
skf = StratifiedKFold(n_splits=10)

In [13]:
index = list(range(len(train_y)))
random.seed(1)
random.shuffle(index)
train_index = index[:20000]
test_index = index[20000:]

In [14]:
x_train = train_data_x[train_index]
y_train = train_data_y[train_index]
x_test = train_data_x[test_index]
y_test = train_data_y[test_index]

### define accuracy

In [15]:
def Accuracy(predict, label):
    total = len(predict)
    trueCount = 0
    for i in range(total):
        if predict[i] == label[i]:
            trueCount += 1
    return  float(trueCount) / float(total)

In [16]:
accList = []
for train, validation in skf.split(x_train, y_train):
    clf.fit(x_train[train], y_train[train])
    predict = clf.predict(x_train[validation])
    accList.append(Accuracy(predict, y_train[validation].tolist()))

In [17]:
avgAcc = sum(accList) / len(accList)
print (avgAcc)

0.868599443524861


In [18]:
clf.fit(x_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

### accuracy on validation data

In [19]:
predictTest = clf.predict(x_test)

In [20]:
Accuracy(predictTest, y_test.tolist())

0.8728

### modify test data

In [21]:
test = pd.read_pickle('E:\\school\\2-2\\AI\\Project2\\data\\Modified_TD_x.pkl')
test.head()

0    [[naturally, film, main, themes, mortality, no...
1    [[movie, disaster, within, disaster, film], [f...
2    [[movie, kids], [saw, tonight, child, loved], ...
3    [[afraid, dark, left, impression, several, dif...
4    [[accurate, depiction, small, time, mob, life,...
Name: review, dtype: object

In [22]:
test_data_x = []
index2word_set = set(model.wv.index2word)
for review in test:
    cnt = 0
    vec = np.zeros((1, 100))
    for sentence in review:
        for word in sentence:
            if word in index2word_set:
                vec = vec + model.wv.__getitem__(word)
                cnt = cnt+1
    vec = vec/cnt
    test_data_x.extend(vec)

In [23]:
test_data_x = np.array(test_data_x)

### predict test y

In [24]:
test_data_y = clf.predict(test_data_x)

In [25]:
test_data_y

array([1, 0, 1, ..., 0, 1, 0], dtype=int64)

### save result

In [26]:
# test_id = pd.read_csv('E:\\school\\2-2\\AI\\Project2\\data\\testData.tsv', sep="\t", header=0)

In [27]:
# test_y_path = 'E:\\school\\2-2\\AI\\Project2\\data\\w2vSVM.csv'

In [28]:
# test_id['sentiment'] = test_data_y

In [29]:
# test_id.to_csv(test_y_path, columns=['id', 'sentiment'], index=0)